In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import nest_asyncio
nest_asyncio.apply()

from deepagents import create_deep_agent
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0,
)
sub_model = ChatOllama(
    model="qwen2.5:latest",
    temperature=0,
)

In [4]:
import httpx

async def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
):
    """Use this to get current exchange rate.

    Args:
        currency_from: The currency to convert from (e.g., "USD").
        currency_to: The currency to convert to (e.g., "EUR").

    Returns:
        A dictionary containing the exchange rate data, or an error message if the request fails.
    """    
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                "https://api.frankfurter.app/latest",
                params={"from": currency_from, "to": currency_to},
            )
            response.raise_for_status()

            data = response.json()
            if "rates" not in data:
                return {"error": "Invalid API response format."}
            return data
    except httpx.HTTPError as e:
        return {"error": f"API request failed: {e}"}
    except ValueError:
        return {"error": "Invalid JSON response from API."}

In [7]:
currency_exchange_prompt = """
You are a helpful assistant that can convert currency from one to another.
Use the tool provided to convert one currency to another.
"""

currency_exchange_sub_agent = {
    "name": "currency_exchange_agent",
    "description": "An agent that can convert currency from one to another in any amount.",
    "prompt": currency_exchange_prompt,
    "tools": ["get_exchange_rate"],
    "model_settings": {"model": sub_model}
}

In [13]:
lead_instructions = """You are an expert data scientist working in a bank. 

Your job is to answer users' questions about currency exchange rates.

Use the currency exchange agent to search for exchange rates. 

You have access to a few tools.

## `get_exchange_rate`

Use this to get the exchange rates for queries relating to exchange rates. 
"""

In [14]:
agent= create_deep_agent(
    [get_exchange_rate],
    lead_instructions,
    subagents=[currency_exchange_sub_agent],
    model=model
)

In [16]:
async for chunk in agent.astream(
    {
        "messages": [
            {"role": "user", "content": "What is the exchange rate from SGD to EUR, USD, and JPY?"}
        ]
    },
    stream_mode="values"
):
    if "messages" in chunk:
        chunk['messages'][-1].pretty_print()

================================ Human Message =================================

What is the exchange rate from SGD to EUR, USD, and JPY?
================================== Ai Message ==================================
Tool Calls:
  write_todos (f76edfd3-90b7-4f39-91e7-51b59ea501ad)
 Call ID: f76edfd3-90b7-4f39-91e7-51b59ea501ad
  Args:
    todos: [{'id': '1', 'status': 'pending', 'task': 'Fetch exchange rates from SGD to EUR, USD, JPY'}]
================================= Tool Message =================================
Name: write_todos

Error: 1 validation error for write_todos
todos.0.content
  Field required [type=missing, input_value={'id': '1', 'status': 'pe...m SGD to EUR, USD, JPY'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  write_todos (9cdf285d-8527-49f2-84da-15c547c1b7c2)
 Call ID: 9cdf285d-8527-49f2-84da